In [1]:
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('./data/news_data.csv', encoding='ISO-8859-1')
df = data.copy()
df.sample(5)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
1654,2006-10-12,0,Harry Pearson on how to get back at North Korea,"Look out, low flying Harrier overhead",Rugby union: Nigel Melville on his American ch...,Tennis: LTA hoping to enlist big name coaches,The question: What next for David Beckham?,How tagging passengers could improve airport s...,Investors see red after iSoft rewards director...,David Conn: children's homes are getting better,...,Leader: In praise of... Vladimir Kramnik,Fiachra Gibbons: This ignorant act will only f...,Stuart Jeffries: Adventures with a cyberman he...,Response: Online poker bosses were only ever a...,"Polly Toynbee: Business, stop moaning. These '...",Richard Norton-Taylor: For their eyes only,Simon Jenkins: Britain has at last escaped the...,Simon Tisdall: World briefing,Mark Lawson: Network premiers,Michael White: No rocking the boat allowed
3081,2012-06-13,0,Former British PM John Major testifies that Ru...,N.Korea's missile launchers came from China: J...,Gay pageant winner beheaded | News24,Norwegian party wants to ban religious male ci...,"German police release photo of mystery boy, wh...",Europe to build world's biggest telescope powe...,Zeta cartel leader caught living in America. M...,Russian President Vladimir Putin vowed yesterd...,...,Police fires warning shots when Polish and Rus...,Photographer Carlo Gianferro photographs inter...,"Mogadishu, the Somali capital, is losing 'Worl...",Thirty percent of Swedes drop out of high scho...,Former Prime Minister John Major said Tuesday ...,UN officially declares Syria in Civil War,\nGreeks hoard cash ahead of key Sunday vote\n...,"""Pope Benedict XVI is on a course correction. ...",Iran draws up designs for nuclear powered subm...,Venezuelan President Hugo Chavez has presented...
1025,2004-04-01,0,The X-Philes: No.37,Matthew Engel: The wrong sort of cheerleaders ...,Fossett on course to smash round-the-world rec...,Wales want Johnson to stay,Linux Loyalists Leery,Sweden 1 - 0 England,Queiroz set for second year at Real,Estonia 0 - 1 Northern Ireland,...,Top 40 downloaded tracks January-March 2004,The Informer: Hughes resigns as immigration mi...,'America's not going anywhere. Not until the j...,Sun threatened with royal boycott,Win sets of classic HG Wells books,West Indies v England - first session,'Strong evidence' of collusion in Ulster killings,Vet schools to introduce admissions tests,Tensions that start in playground,Jon Henley on the French cabinet reshuffle
3364,2013-07-31,0,Snowden's father says FBI asked him to fly to ...,"""Egypt's cultural heritage is in danger. Grave...",To give some perspective to the recent story c...,"In Turkey, Higher Education Loans and Dormitor...",XKeyscore: NSA tool collects 'nearly everythin...,Woolly mammoth DNA may lead to a resurrection ...,Saudi princess to face trial in US court,Military judge finds Bradly Manning not guilty...,...,"""There's no question I have access to informat...",Long-living Japanese society needs better qual...,Global warmings new natural disaster: Himalaya...,London police officers face action over woman'...,The father of US fugitive Edward Snowden says ...,"After Toronto street car shooting, Canadian po...",Why fewer ground reports are emerging from Syr...,Cardinal Dolan Clarifies Pope Francis Remarks:...,U.S. wants exemption from Canadian law for cro...,UN gives DR Congo rebels 48 hours to hand in w...
1404,2005-10-03,1,"Football: Scouting report: Boaz Myhill, Hull City",Football: Keane a candidate for Ireland job,"Football: Ecclestone wanted to buy Chelsea, sa...",Football: Cole wants Campbell beside him,Football: Candid Cole prepared to bear Chelsea...,Football: Miller puts out feelers to Hearts,Football: Arsenal fret over Henry's refusal to...,Football: Carsley setback adds to Everton's woes,...,Police storm Gaza legislature in protest at Ha...,Column five: The house that became a war zone,Girl who tied boy 

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4101 entries, 0 to 4100
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    4101 non-null   object
 1   Label   4101 non-null   int64 
 2   Top1    4101 non-null   object
 3   Top2    4101 non-null   object
 4   Top3    4101 non-null   object
 5   Top4    4101 non-null   object
 6   Top5    4101 non-null   object
 7   Top6    4101 non-null   object
 8   Top7    4101 non-null   object
 9   Top8    4101 non-null   object
 10  Top9    4101 non-null   object
 11  Top10   4101 non-null   object
 12  Top11   4101 non-null   object
 13  Top12   4101 non-null   object
 14  Top13   4101 non-null   object
 15  Top14   4101 non-null   object
 16  Top15   4101 non-null   object
 17  Top16   4101 non-null   object
 18  Top17   4101 non-null   object
 19  Top18   4101 non-null   object
 20  Top19   4101 non-null   object
 21  Top20   4101 non-null   object
 22  Top21   4101 non-null   

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Combine all topic columns into a single text column
df['text'] = df[['Top' + str(i) for i in range(1, 26)]].fillna('').agg(' '.join, axis=1)

# Drop the individual topic columns and keep only 'Date', 'Label', and 'text'
df = df[['Date', 'Label', 'text']]

# Preprocess text data
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)  # Remove all non-word characters
    text = text.lower()  # Convert to lowercase
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['text'] = df['text'].apply(preprocess_text)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['Label'], test_size=0.2, random_state=42)

# Create a pipeline with TF-IDF and RandomForestClassifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=2000, ngram_range=(1, 2))),
    ('clf', LogisticRegression(random_state=42, max_iter=200))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

[nltk_data] Downloading package stopwords to /home/oem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/oem/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


              precision    recall  f1-score   support

           0       0.44      0.41      0.42       372
           1       0.54      0.57      0.55       449

    accuracy                           0.50       821
   macro avg       0.49      0.49      0.49       821
weighted avg       0.49      0.50      0.49       821



In [6]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Initialize VADER sentiment intensity analyzer
analyzer = SentimentIntensityAnalyzer()

# Apply VADER to each text entry to get compound scores
def get_vader_sentiment(text):
    score = analyzer.polarity_scores(text)['compound']
    # Define thresholds to classify as positive, neutral, or negative based on your Label encoding
    return 1 if score >= 0.01 else 0

df = data.copy()
# Assuming 'text' column has the combined topic text and 'Label' column is your target
df['text'] = df[['Top' + str(i) for i in range(1, 26, 6)]].fillna('').agg(' '.join, axis=1)

# Preprocess text data
# nltk.download('stopwords')
# nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)  # Remove all non-word characters
    text = text.lower()  # Convert to lowercase
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['text'] = df['text'].apply(preprocess_text)

df['vader_sentiment'] = df['text'].apply(get_vader_sentiment)

# Split data
X_train, X_test, y_train, y_test = train_test_split(df['vader_sentiment'], data['Label'], test_size=0.2, random_state=42)

# Evaluate performance
print(classification_report(y_test, X_test))


              precision    recall  f1-score   support

           0       0.47      0.73      0.57       372
           1       0.59      0.33      0.42       449

    accuracy                           0.51       821
   macro avg       0.53      0.53      0.50       821
weighted avg       0.54      0.51      0.49       821



In [7]:
df.sample(5)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,text,vader_sentiment
3348,2013-07-09,1,Hero Fukushima ex-manager who foiled nuclear d...,Majority of people worldwide believe corruptio...,New Edward Snowden video interview released,Ireland's High Court refuses to grant arrest w...,How The Guardian Broke the Snowden Story ... a...,"Snowden chooses Venezuela for asylum, agrees t...","Warren Buffett just donated 22,870,529 shares ...",Fukushima groundwater now 90x more radioactive...,...,Brazil Opens Investigation into US Spying,22 Staffers Resign At Al-Jazeera Over Claims O...,"Toronto battered by storm, flooding; 300,000 r...",Second 'Prisoner X' held in top secret in Israel,Venezuela receives formal asylum request from ...,A Spanish newspaper has published what it alle...,Revealed: NSA Put Latin American Countries in ...,Pollution in northern China has led to drop of...,hero fukushima ex manager foiled nuclear disas...,1
57,2000-03-27,1,Wilkinson backs plan for qualified managers,McIlroy looks to outsmart Maltese with crosses,Absentees pick Brown's team,Banks consider ATM phone top-ups,Anarchists at play,Painful requiems,Inhale those vibes,Expansive QXL snaps up auction rival,...,First Division round-up,A loud appeal for Lara's return,Relentless Waugh feeds record-breaking habit,Blues to lose Babayaro for Olympics,Undaunted Watford still high on believing,Sky blues fall in on Ogrizovic,Shreeves mixes his semaphores,Phillips signs up with solo sensation,wilkinson back plan qualified manager inhale v...,0
171,2000-09-11,0,Sparta Prague 0 - 1 Arsenal,Scorn for euro ministers,Pop review: Neko Case and Her Boyfriends,GE 'bids for Scottish Life',Deutsche Bank blamed as shares hit by rogue trade,What a song and dance,Purity and smut,Elderly in savings trap to get cash boost,...,City owe it all to Styles not substance,Bargains underpin O'Neill record run,Bad break puts pressure on Ellis,Surrey count down to glory,Hamann the man in team of two halves,Derby cede comeback crown,Cole fires another rescue mission but Arsenal ...,Card chaos but Fulham hold the aces,sparta prague 0 1 arsenal purity smut glamorga...,0
2167,2008-10-27,0,b'BBC: Rampant Evidence That Georgia Committed...,b'Is Britain on its way to becoming a surveill...,b'Extremists to be barred from UK',b'1000 Congolese citizens hurl rocks at a UN c...,b'Happy Diwali ',b'UK parents to be fingerprinted by nursery sc...,b'Europe on the brink of currency crisis meltd...,"b'Defectors describe executions, torture in No...",...,"b'IMF may need to ""print money"" as crisis spre...","b""Refugees Australia rejected 'killed by Talib...","b""'Bush's Way of Waving Goodbye to Syria?'""",b'UN atomic energy chief warns of nuclear theft ',b'China May Lend Russia $25 Billion as Part of...,"b'Q&amp;A: Why is Congo in the news, and what ...","b""I don't get it? Are people who insist on war...","b""Iranian general admits supplying weapons to ...",b bbc rampant evidence georgia committed war c...,0
970,2004-01-13,1,Larry Ellison weds again,Who goes where?,Seaman calls it quits,City Hammer out James deal,Finance ministers face legal action,2 for 1 flights to the Americas,Capital confections,Who goes where?,...,Cutbacks mean students leave university less w...,CRM - a jargon-buster,Bank of England officials 'turned away' from B...,"Gene, King Tut's Wah Wah Hut, Glasgow",BAE snubbed in air tanker contract,Honours row 'real blow' for UK science,When are we most vulnerable?,House of horrors,larry ellison wed capital confection granada g...,0
